## Azure AI Foundry Agent Service: Logic App (Standard) tools

This notebook demonstrates the setup and use of Azure Standard-type Logic Apps as a tool for Azure AI Foundry's Agent Service, using OpenAPI schema integration.

### Environment Setup

In [1]:
# Import required packages
import os
from typing import Dict, Any
from urllib.parse import urlparse, parse_qs

from azure.identity import DefaultAzureCredential
from azure.mgmt.web import WebSiteManagementClient 

from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    OpenApiTool,
    OpenApiManagedAuthDetails,
    OpenApiManagedSecurityScheme
)

In [2]:
# Set environment variables
PROJECT_ENDPOINT = os.environ.get("AZURE_FOUNDRY_PROJECT_ENDPOINT")
MODEL_DEPLOYMENT = os.environ.get("AZURE_FOUNDRY_GPT_MODEL")
SUBSCRIPTION_ID = os.environ.get("AZURE_SUBSCRIPTION_ID")
RESOURCE_GROUP = os.environ.get("RESOURCE_GROUP_NAME", "App_LogicApp")

LOGIC_APP_NAME = "Laziz-StandardLA"     # Your Standard Logic App name
WORKFLOW_NAME = "lazizdemologicapp"     # Your workflow inside the Standard Logic App
TRIGGER_NAME = "HTTP-trigger-standard"  # Your HTTP trigger name

### Helper Class - Logic Apps Integration

In [3]:
# Logic App Integration class for Standard Logic Apps with Easy Auth
class StandardLogicAppsIntegration:
    """
    Integration class for Standard Logic Apps using Easy Auth (Managed Identity).
    """
    
    def __init__(self, subscription_id: str, resource_group: str, credential=None):
        if credential is None:
            credential = DefaultAzureCredential()
        
        self.subscription_id = subscription_id
        self.resource_group = resource_group
        self.credential = credential
        self.web_client = WebSiteManagementClient(credential, subscription_id)
    
    def get_callback_url(self, logic_app_name: str, workflow_name: str, trigger_name: str) -> str:
        """Get the callback URL for a Standard Logic App workflow trigger."""
        try:
            callback = self.web_client.workflow_triggers.list_callback_url(
                resource_group_name = self.resource_group,
                name = logic_app_name,
                workflow_name = workflow_name,
                trigger_name = trigger_name
            )
            
            if callback.value is None:
                raise ValueError(
                    f"No callback URL returned for workflow '{workflow_name}' "
                    f"in Logic App '{logic_app_name}'."
                )
            
            print(f"Retrieved callback URL for workflow: {workflow_name}")
            return callback.value
            
        except Exception as e:
            print(f"Error getting callback URL: {e}")
            raise
    
    def create_direct_openapi_spec(self, callback_url: str, workflow_description: str = None) -> Dict[str, Any]:
        """
        Create OpenAPI spec for Standard Logic App with Easy Auth.
        """
        print("Creating OpenAPI spec with Easy Auth (Managed Identity)...")

        if workflow_description is None:
            workflow_description = "Standard Logic App workflow endpoint with Easy Auth"

        # Parse URL and use ONLY the base path, no SAS query parameters
        parsed_url = urlparse(callback_url)
        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
        
        # print(f"Base URL (without SAS): {base_url}")
        
        openapi_spec = {
            "openapi": "3.0.0",
            "info": {
                "title": "Standard Logic App Weather API",
                "version": "1.0.0",
                "description": "Standard Logic App with Easy Auth managed identity authentication"
            },
            "servers": [{"url": base_url}],
            "paths": {
                "/": {
                    "post": {
                        "operationId": "get_weather",
                        "summary": "Get weather information",
                        "description": workflow_description,
                        # No query parameters - Easy Auth uses Bearer token in header
                        "requestBody": {
                            "required": True,
                            "content": {
                                "application/json": {
                                    "schema": {
                                        "type": "object",
                                        "properties": {
                                            "Location": {
                                                "type": "string",
                                                "description": "The location to get weather for",
                                                "example": "London"
                                            }
                                        },
                                        "required": ["Location"]
                                    }
                                }
                            }
                        },
                        "responses": {
                            "200": {
                                "description": "Weather information",
                                "content": {
                                    "application/json": {
                                        "schema": {
                                            "type": "object",
                                            "description": "Weather forecast data"
                                        }
                                    }
                                }
                            },
                            "202": {
                                "description": "Accepted - workflow started asynchronously",
                                "content": {
                                    "application/json": {
                                        "schema": {
                                            "type": "object"
                                        }
                                    }
                                }
                            },
                            "401": {
                                "description": "Unauthorized - invalid or missing Bearer token"
                            },
                            "403": {
                                "description": "Forbidden - token valid but identity not authorized"
                            }
                        },
                        "security": [
                            {
                                "oauth2": []
                            }
                        ]
                    }
                }
            },
            "components": {
                "securitySchemes": {
                    "oauth2": {
                        "type": "oauth2",
                        "flows": {
                            "clientCredentials": {
                                "tokenUrl": "https://login.microsoftonline.com/common/oauth2/v2.0/token",
                                "scopes": {}
                            }
                        }
                    }
                }
            }
        }

        print("Successfully created OpenAPI spec with Easy Auth")
        return openapi_spec
    
    def create_openapi_tool(self, logic_app_name: str, workflow_name: str, 
                          trigger_name: str, tool_name: str = None,
                          tool_description: str = None) -> tuple[OpenApiTool, str]:
        """
        Create OpenAPI tool for Standard Logic App with Easy Auth.
        """
        callback_url = self.get_callback_url(logic_app_name, workflow_name, trigger_name)
        openapi_spec = self.create_direct_openapi_spec(callback_url, workflow_description=tool_description)
        
        if not tool_name:
            tool_name = "get_weather"
        if not tool_description:
            tool_description = "Get weather forecast for any location via Standard Logic App"
        
        # CRITICAL: Use the App Registration Client ID as the audience
        # This is the clientId from your Easy Auth configuration
        client_id = "170bd90b-ead7-45f1-a809-aa7f9bc446db"  # From your Easy Auth config
        audience = f"api://{client_id}"
    
        auth = OpenApiManagedAuthDetails(
            security_scheme = OpenApiManagedSecurityScheme(
                audience = audience
            )
        )
        
        tool = OpenApiTool(
            name = tool_name,
            spec = openapi_spec,
            description = tool_description,
            auth = auth
        )
        
        print(f"Created OpenAPI tool with Easy Auth: {tool_name}")
        # print(f"Using audience: {audience}")
        return tool, tool_name

print("Standard Logic App Integration class (Easy Auth) successfully defined")

Standard Logic App Integration class (Easy Auth) successfully defined


### Logic Apps Tool Setup

In [4]:
# Initialise Standard Logic App integration
print("Initialising Standard Logic App integration...")
logic_integration = StandardLogicAppsIntegration(
    subscription_id = SUBSCRIPTION_ID,
    resource_group = RESOURCE_GROUP
)

print("Standard Logic App integration initialised successfully")

Initialising Standard Logic App integration...
Standard Logic App integration initialised successfully


In [5]:
# Create OpenAPI Tool for Standard Logic App
print("Creating OpenAPI tool for Standard Logic App workflow...")
openapi_tool, tool_name = logic_integration.create_openapi_tool(
    logic_app_name = LOGIC_APP_NAME,
    workflow_name = WORKFLOW_NAME,
    trigger_name = TRIGGER_NAME,
    tool_name = "get_weather",
    tool_description = "Get weather forecast for any location"
)

print(f"OpenAPI tool created successfully: {tool_name}")

Creating OpenAPI tool for Standard Logic App workflow...
Retrieved callback URL for workflow: lazizdemologicapp
Creating OpenAPI spec with Easy Auth (Managed Identity)...
Successfully created OpenAPI spec with Easy Auth
Created OpenAPI tool with Easy Auth: get_weather
OpenAPI tool created successfully: get_weather


In [6]:
# Debug: Check Easy Auth Configuration
print("Checking Easy Auth configuration for Logic App...")
print(f"Logic App: {LOGIC_APP_NAME}")
print(f"Resource Group: {RESOURCE_GROUP}")
print("-" * 50)

try:
    # Get the auth settings for the Logic App
    auth_settings = logic_integration.web_client.web_apps.get_auth_settings_v2(
        resource_group_name=RESOURCE_GROUP,
        name=LOGIC_APP_NAME
    )
    
    print(f"Auth Settings Type: {type(auth_settings)}")
    print(f"Available attributes: {dir(auth_settings)}")
    print()
    
    # Try to access the settings directly
    print("Easy Auth Status:")
    
    # Check if platform is enabled
    if hasattr(auth_settings, 'platform') and auth_settings.platform:
        print(f"  Platform Enabled: {auth_settings.platform.enabled}")
    
    # Check global validation
    if hasattr(auth_settings, 'global_validation') and auth_settings.global_validation:
        print(f"  Require Authentication: {auth_settings.global_validation.require_authentication}")
        print(f"  Unauthenticated Action: {auth_settings.global_validation.unauthenticated_client_action}")
    
    # Check Azure AD configuration
    if hasattr(auth_settings, 'identity_providers') and auth_settings.identity_providers:
        aad = auth_settings.identity_providers.azure_active_directory
        if aad and aad.enabled:
            print("\nAzure AD Configuration:")
            print(f"  Enabled: {aad.enabled}")
            
            # if hasattr(aad, 'registration') and aad.registration:
            #     print(f"  OpenID Issuer: {aad.registration.open_id_issuer}")
            #     print(f"  Client ID: {aad.registration.client_id}")
            
            # if hasattr(aad, 'validation') and aad.validation:
            #     if hasattr(aad.validation, 'allowed_audiences') and aad.validation.allowed_audiences:
            #         print(f"  Allowed Audiences:")
            #         for aud in aad.validation.allowed_audiences:
            #             print(f"    - {aud}")
                
            #     if hasattr(aad.validation, 'default_authorization_policy') and aad.validation.default_authorization_policy:
            #         policy = aad.validation.default_authorization_policy
            #         if hasattr(policy, 'allowed_principals') and policy.allowed_principals:
            #             if hasattr(policy.allowed_principals, 'identities') and policy.allowed_principals.identities:
            #                 print(f"  Allowed Identities (Object IDs):")
            #                 for identity in policy.allowed_principals.identities:
            #                     print(f"    - {identity}")
        else:
            print("\nAzure AD: Not enabled or not found")
    else:
        print("\nNo identity providers found")
    
    # print("-" * 50)
    # print("\nRAW AUTH SETTINGS (for debugging):")
    # print(auth_settings)
    
except AttributeError as ae:
    print(f"Attribute error: {ae}")
    print("\nTrying alternative access method...")
    
    try:
        # Try accessing as dictionary
        auth_dict = auth_settings.as_dict() if hasattr(auth_settings, 'as_dict') else None
        if auth_dict:
            import json
            print(json.dumps(auth_dict, indent=2))
    except Exception as e2:
        print(f"Could not convert to dict: {e2}")
        
except Exception as e:
    print(f"Error checking auth settings: {e}")
    print(f"Error type: {type(e)}")

# print("\n" + "=" * 50)
# print("AI Foundry Project Information:")
# print(f"Project Endpoint: {PROJECT_ENDPOINT}")

Checking Easy Auth configuration for Logic App...
Logic App: Laziz-StandardLA
Resource Group: App_LogicApp
--------------------------------------------------
Auth Settings Type: <class 'azure.mgmt.web.models._models_py3.SiteAuthSettingsV2'>
Available attributes: ['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_attribute_map', '_classify', '_create_xml_node', '_flatten_subtype', '_get_rest_key_parts', '_infer_class_models', '_subtype_map', '_validation', 'additional_properties', 'as_dict', 'deserialize', 'enable_additional_properties_sending', 'from_dict', 'global_validation', 'http_settings', 'id', 'identity_providers', 'is_xml_model', 'kind', 'login', 'name'

### AI Agent Setup

In [7]:
# Initialise AI Project Client
print("Initialising AI Project client...")
project_client = AIProjectClient(
    endpoint = PROJECT_ENDPOINT,
    credential = DefaultAzureCredential(),
)

agents_client = project_client.agents
print("AI Project client initialised successfully")

Initialising AI Project client...
AI Project client initialised successfully


In [8]:
# Create AI Agent
print("Creating weather agent...")
agent = agents_client.create_agent(
    model = MODEL_DEPLOYMENT,
    name = "standardlogicapp-weather-agent",
    instructions = "You are a helpful weather assistant. When asked about weather, use the get_weather tool with the location provided by the user.",
    tools = openapi_tool.definitions,
)

print(f"Created agent successfully, ID: {agent.id}")

Creating weather agent...
Created agent successfully, ID: asst_7iUkzyeipTnldjsWsiPXCL4b


In [9]:
# Create Thread
print("Creating conversation thread...")
thread = agents_client.threads.create()

print(f"Created thread successfully, ID: {thread.id}")

Creating conversation thread...
Created thread successfully, ID: thread_tr25EVHKNQUzkgwnn5RYLOxl


In [10]:
# Create Message and Run
print("Creating message and running agent...")
message = agents_client.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "What's the weather in London?",
)

print(f"Created message, ID: {message.id}")

# Create and process run
run = agents_client.runs.create_and_process(
    thread_id = thread.id,
    agent_id = agent.id
)

print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Creating message and running agent...
Created message, ID: msg_Tb4y0Ao3GYFQoEQlBPorerLD
Run finished with status: RunStatus.COMPLETED


In [11]:
# Display Results
print("Displaying conversation results...")
messages = agents_client.messages.list(thread_id=thread.id)
messages_list = list(messages)
print(f"Found {len(messages_list)} messages")
print("-" * 50)

for i, msg in enumerate(reversed(messages_list)):  # Reverse to show chronological order
    role = msg.role.capitalize()
    
    if hasattr(msg, 'content') and msg.content:
        try:
            if hasattr(msg.content[0], 'text') and hasattr(msg.content[0].text, 'value'):
                content = msg.content[0].text.value
            else:
                content = str(msg.content[0])
        except (IndexError, AttributeError):
            content = "No readable content"
    else:
        content = "No content"
    
    print(f"{role}: {content}")
    print("-" * 50)

Displaying conversation results...
Found 3 messages
--------------------------------------------------
User: What's the weather in London?
--------------------------------------------------
Assistant: It seems I encountered an issue while trying to get the weather for London. Let me try again.
--------------------------------------------------
Assistant: I am currently experiencing technical difficulties retrieving the weather information for London. Please try again later. If you want, I can help with something else!
--------------------------------------------------


### Housekeeping

In [12]:
# Delete the thread and agent when finished
try:
    # Delete the thread
    agents_client.threads.delete(thread.id)
    print("Deleted thread successfully")

    # Use the existing agents_client (no need for 'with' block)
    agents_client.delete_agent(agent.id)
    print("Deleted weather agent successfully")
    
    # Close the project client
    project_client.close()
    print("Closed project client connection")
    
except Exception as e:
    print(f"Error during cleanup: {e}")

print("Cleanup complete!")

Deleted thread successfully
Deleted weather agent successfully
Closed project client connection
Cleanup complete!
